In [6]:
from utils.model import *
from utils.data import *
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
max_values_df = get_hf_max_values("../data/hf-max_values.pkl.gzip")

Example of online learning on SoyKB dataset

In [ ]:
dataset = "soykb"

with open(f"dataset/{dataset}-split_75_0_25.static", "rb") as f:
    [train, _, test] = pickle.load(f)

scaler = MaxAbsScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

In [ ]:
step = 10
n_past = 5 * 60
n_future = 5 * 60

features = (0, 1, 2, 3, 6, 9, 11)
n_features = len(features)
n_static = 8
features_pred = (1, 2)
n_features_pred = len(features_pred)
split_stride = 1

In [ ]:
with open(f"dataset/{dataset}-split_75_0_25-step_{step}s-past_{n_past}s-future_{n_future}s.dynamic", "rb") as f:
    [train_series, _, test_series] = pickle.load(f)

series_scaler = MaxAbsScaler()
series_scaler.fit([max_values_df.loc[train.index].max()])
train_series_scaled = [series_scaler.transform(serie) for serie in train_series]
test_series_scaled = [series_scaler.transform(serie) for serie in test_series]

X_train, y_train = dataset_split_many_2_many(train_scaled, train_series_scaled, n_past//step, n_future//step, features, features_pred)
X_test, y_test = dataset_split_many_2_many(test_scaled, test_series_scaled, n_past//step, n_future//step, features, features_pred)

model = example_encoder_decoder_model(
  n_features = n_features,
  n_features_pred = n_features_pred,
  n_static = n_static,
  n_past = (n_past//step),
  n_future = (n_future//step),
)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss="mse")


train_indexes = np.random.permutation(range(len(train)))

for train_index in train_indexes:
    X = X_train[train_index]
    y = y_train[train_index]
    samples = y.shape[0]
    model.fit(
      X,
      y,
      epochs=1,
      batch_size=1,
      verbose=1,
      callbacks=[]
    )

    # restart states after fitting each of the sequences
    model.reset_states()